In [1]:
# Khai báo các thư viện cần thiết
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Pandas tùy chỉnh
pd.set_option('display.max_colwidth', 1000, 'display.max_rows', None, 'display.max_columns', None)

# Plotting tùy chỉnh
%matplotlib inline
mpl.style.use('ggplot')

# Đọc dữ liệu từ file csv
transactions_total = pd.read_csv('C:/Users/green/Downloads/DemoDA3/creditcard.csv')

In [3]:
# Chọn lọc dữ liệu
transactions_1 = transactions_total[transactions_total['Class'] == 1] # Lấy ra các giao dịch bất thường
transactions_0 = transactions_total[transactions_total['Class'] == 0] # Lấy ra các giao dịch bình thường
print('Trong dataset, có ' + str(len(transactions_1)) +" giao dịch bất thường do đó chúng ta sẽ lấy số lượng tương tự đối với các giao dịch bình thường")

transactions_temp = transactions_0.sample(500)
transactions = transactions_1.append(transactions_temp) # Gộp các giao dịch bất thường và bình thường với nhau 
transactions = transactions.sample(frac=1) # Trộn thử tự các record của dataset

transactions.info()

Trong dataset, có 492 giao dịch bất thường do đó chúng ta sẽ lấy số lượng tương tự đối với các giao dịch bình thường
<class 'pandas.core.frame.DataFrame'>
Int64Index: 992 entries, 187949 to 17453
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    992 non-null    float64
 1   V1      992 non-null    float64
 2   V2      992 non-null    float64
 3   V3      992 non-null    float64
 4   V4      992 non-null    float64
 5   V5      992 non-null    float64
 6   V6      992 non-null    float64
 7   V7      992 non-null    float64
 8   V8      992 non-null    float64
 9   V9      992 non-null    float64
 10  V10     992 non-null    float64
 11  V11     992 non-null    float64
 12  V12     992 non-null    float64
 13  V13     992 non-null    float64
 14  V14     992 non-null    float64
 15  V15     992 non-null    float64
 16  V16     992 non-null    float64
 17  V17     992 non-null    float64
 18  V18     992 non-null    fl

In [4]:
transactions.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
187949,127761.0,2.238816,-1.387430,-0.787411,-1.473788,-1.355071,-0.727917,-1.114592,-0.185003,-1.204946,1.543282,-0.648592,-0.651120,0.398683,-0.435511,-0.120042,-0.697132,0.662524,-0.304334,-0.341277,-0.400199,-0.161543,-0.003115,0.335745,1.063670,-0.287435,-0.256300,0.011713,-0.035420,30.00,0
135034,81079.0,1.118055,-0.114411,0.519357,1.102106,-0.344508,0.385339,-0.389015,0.361145,0.607521,0.021051,0.516438,-0.056129,-2.443876,0.607243,-0.324819,-0.159664,-0.032878,-0.107344,0.118952,-0.300695,-0.235219,-0.625905,0.041126,-0.381279,0.356547,-0.505420,0.035960,0.007233,12.99,0
111690,72327.0,-4.198735,0.194121,-3.917586,3.920748,-1.875486,-2.118933,-3.614445,1.687884,-2.189871,-4.684233,4.376907,-5.007441,-1.304745,-6.192475,1.002404,-3.316934,-6.188834,-1.040413,1.233044,1.003350,0.801312,-0.183001,-0.440387,0.292539,-0.144967,-0.251744,1.249414,-0.131525,238.90,1
261166,159892.0,-2.403431,-0.110511,0.493498,0.340880,-0.916951,0.459596,-0.454698,1.142350,-1.656750,0.469991,0.096155,-0.020770,-0.654112,0.869286,0.053409,-1.323993,0.262321,2.252322,0.000891,-0.095678,-0.208971,-0.504770,-0.263229,0.637831,0.554328,-0.403327,0.288518,-0.114721,150.00,0
18466,29526.0,1.102804,2.829168,-3.932870,4.707691,2.937967,-1.800904,1.672734,-0.300240,-2.783011,-1.884842,3.792565,-1.460471,-1.312776,-7.077361,-0.299466,2.493228,6.244987,2.971749,-2.679950,-0.030880,-0.106994,-0.250050,-0.521627,-0.448950,1.291646,0.516327,0.009146,0.153318,0.68,1


In [5]:
# Phân tách dataset
X = transactions.drop(labels='Class', axis=1) # Features
y = transactions.loc[:,'Class']               # Response
del transactions                              # Xóa dữ liệu lúc đầu

# Tách dataset thành 80% cho training và 20% cho test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
del X, y

In [6]:
from sklearn import svm

classifier = svm.SVC(kernel='linear') # Thiết lập SVM classifier

In [7]:
classifier.fit(X_train, y_train) # Train model

SVC(kernel='linear')

In [8]:
# Dự đoán cho bộ dữ liệu test
y_pred = classifier.predict(X_test)

In [9]:
# Kết quả dự đoán
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[98  2]
 [28 71]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.98      0.87       100
           1       0.97      0.72      0.83        99

    accuracy                           0.85       199
   macro avg       0.88      0.85      0.85       199
weighted avg       0.87      0.85      0.85       199

Accuracy: 0.8492462311557789
